In [1]:
from nlb_tools.nwb_interface import NWBDataset
from nlb_tools.make_tensors import make_train_input_tensors, make_eval_input_tensors

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from models import HMM


dir = '/Users/akshita/Documents/Acads/695GM/project/000140/sub-Jenkins'
# dataset_train = NWBDataset(dir, "*train", split_heldout=True)
# dataset_test = NWBDataset(dir, "*test", split_heldout=True)
dataset = NWBDataset(dir)

dataset.resample(5)
dataset.bin_width

train_dict = make_train_input_tensors(dataset=dataset, 
                                      dataset_name='mc_maze_small', 
                                      trial_split='train', # trial_split=['train', 'val'], for Test phase
                                      save_file=False, 
                                      include_forward_pred=True)

eval_dict = make_eval_input_tensors(dataset=dataset,
                                    dataset_name='mc_maze_small',
                                    trial_split='val', # trial_split='test', for Test phase
                                    save_file=False)

training_input = torch.Tensor(
    np.concatenate([
        train_dict['train_spikes_heldin'], 
        np.zeros(train_dict['train_spikes_heldin_forward'].shape), # zeroed inputs for forecasting
    ], axis=1)).long()

training_output = torch.Tensor(
    np.concatenate([
        np.concatenate([
            train_dict['train_spikes_heldin'],
            train_dict['train_spikes_heldin_forward'],
        ], axis=1),
        np.concatenate([
            train_dict['train_spikes_heldout'],
            train_dict['train_spikes_heldout_forward'],
        ], axis=1),
    ], axis=2)).long()

eval_input = torch.Tensor(
    np.concatenate([
        eval_dict['eval_spikes_heldin'],
        np.zeros((
            eval_dict['eval_spikes_heldin'].shape[0],
            train_dict['train_spikes_heldin_forward'].shape[1],
            eval_dict['eval_spikes_heldin'].shape[2]
        )),
    ], axis=1)).long()


In [4]:

# from data import get_datasets, read_config
# from training import Trainer

# Generate datasets from text file
# path = "data"
# N = 128
# config = read_config(N,path)
# train_dataset, valid_dataset = get_datasets(config)
# checkpoint_path = "."

# config = 

M = 5
N = 107
out_dim = 142
T = torch.ones(75, dtype=torch.long) * 180

# Initialize model
model = HMM(M, N, out_dim)

model(training_input[:,:, 0].squeeze(-1), T)

# Train the model
# num_epochs = 10
# trainer = Trainer(model, config, lr=0.003)
# trainer.load_checkpoint(checkpoint_path)

# for epoch in range(num_epochs):
# 	print("========= Epoch %d of %d =========" % (epoch+1, num_epochs))
# 	train_loss = trainer.train(train_dataset)
# 	valid_loss = trainer.test(valid_dataset)
# 	trainer.save_checkpoint(epoch, checkpoint_path)

# 	print("========= Results: epoch %d of %d =========" % (epoch+1, num_epochs))
# 	print("train loss: %.2f| valid loss: %.2f\n" % (train_loss, valid_loss) )




l_sp: torch.Size([107])
l_alpha: torch.Size([75, 180, 107])
emis_inp:  torch.Size([75])
emis_out:  torch.Size([75, 107])
tensor([[-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799],
        [-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799],
        [-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799],
        ...,
        [-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799],
        [-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799],
        [-7.1742, -8.5736, -6.5210,  ..., -8.2924, -8.1644, -6.4799]],
       grad_fn=<SliceBackward0>)
emis_inp:  torch.Size([75])
emis_out:  torch.Size([75, 107])


NameError: name '_genbmm' is not defined